In [1]:
import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [2]:
lemmatizer = WordNetLemmatizer()

intents = json.loads(open('intents.json').read())

words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

In [3]:
import nltk
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to C:\Users\Syauqi
[nltk_data]     Nabil\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
         word_list = nltk.word_tokenize(pattern)
         words.append(word_list)
         documents.append((word_list, intent['tag']))
         if intent['tag'] not in classes:
              classes.append(intent['tag'])

In [5]:
for word in words:
    if isinstance(word, list):
        print("Sublist ditemukan dalam words:", word)

Sublist ditemukan dalam words: ['Hello', ',', 'Stella']
Sublist ditemukan dalam words: []
Sublist ditemukan dalam words: ['ohayou']
Sublist ditemukan dalam words: ['aku', 'mau', 'minta', 'tolong', 'dong']
Sublist ditemukan dalam words: ['boleh', 'minta', 'tolong', 'gak', '?']
Sublist ditemukan dalam words: ['makasih', 'ya']
Sublist ditemukan dalam words: ['terima', 'kasih']
Sublist ditemukan dalam words: ['oke']
Sublist ditemukan dalam words: ['bye']
Sublist ditemukan dalam words: ['makasih']
Sublist ditemukan dalam words: ['iya', ',', 'ntar', 'ku', 'chat', 'lagi']
Sublist ditemukan dalam words: ['nanti', 'aku', 'chat', 'lagi', 'deh']
Sublist ditemukan dalam words: ['What', 'is', 'progamming', '?']
Sublist ditemukan dalam words: ['What', 'is', 'coding', '?']
Sublist ditemukan dalam words: ['Tell', 'me', 'about', 'programming']
Sublist ditemukan dalam words: ['Tell', 'me', 'about', 'coding']
Sublist ditemukan dalam words: ['What', 'is', 'software', 'development', '?']
Sublist ditemukan 

In [6]:
words = [word for sublist in words for word in sublist]

words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

print(words)

['Are', 'Can', 'Check', 'Do', 'Explain', 'Flask', 'Hello', 'How', 'Stella', 'Summarize', 'Tell', 'What', 'about', 'aja', 'aku', 'bagaimana', 'baik', 'boleh', 'btw', 'bye', 'chat', 'check', 'coding', 'deh', 'development', 'ditanya', 'do', 'dong', 'feel', 'gak', 'gimana', 'good', 'is', 'iya', 'juga', 'kabar', 'kamu', 'kan', 'kasih', 'know', 'ku', 'lagi', 'letter', 'makasih', 'mau', 'me', 'minta', 'nanti', 'nih', 'ntar', 'number', 'ohayou', 'oke', 'please', 'prime', 'progamming', 'programming', 'software', 'something', 'summarize', 'tanya', 'tell', 'terima', 'text', 'this', 'to', 'tolong', 'ya', 'you']


In [7]:
classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [8]:
training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

In [9]:
# expected_bag_length = len(words)  # Harus sama dengan panjang 'words'
# expected_output_row_length = len(classes)  # Harus sama dengan panjang 'classes'

# # Pastikan setiap elemen dalam 'training' memiliki bentuk yang konsisten
# for i, (bag, output_row) in enumerate(training):
#     if len(bag) != expected_bag_length or len(output_row) != expected_output_row_length:
#         print(f"Inconsistent shape at item {i}: Bag length: {len(bag)}, Output row length: {len(output_row)}")

In [10]:
random.shuffle(training)
training = np.array(training, dtype=object)
# training = np.array(training)

train_x = np.array([item[0] for item in training])
train_y = np.array([item[1] for item in training])

# train_x = list(training[:, 0])
# train_y = list(training[:, 1])

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]), ), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.5, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('Stella-model.h5', hist)
print("Done")

Epoch 1/200


c:\Users\Syauqi Nabil\Stella\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\Syauqi Nabil\Stella\.venv\Lib\site-packages\keras\src\optimizers\base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0958 - loss: 2.8213
Epoch 2/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0567 - loss: 2.8249.90
Epoch 3/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 909us/step - accuracy: 0.0492 - loss: 2.8646
Epoch 4/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0153 - loss: 2.7660      
Epoch 5/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 909us/step - accuracy: 0.1509 - loss: 2.7586
Epoch 6/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 909us/step - accuracy: 0.1123 - loss: 2.7077 
Epoch 7/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0471 - loss: 2.7589    
Epoch 8/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1107 - loss: 2.7762
Epoch 9/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 819us/step - accuracy: 0.0538 - loss: 2.7561    
Epoch 10/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 864us/step - accuracy: 0.0357 - loss: 2.7502    
Epoch 11/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1347 - loss: 2.6882  
Epoch 12/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0

Done
